https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_split.ipynb

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
transaction = pd.read_csv('book_transactions.csv', encoding='cp949')

In [3]:
id2user = dict(enumerate(transaction.회원번호.unique()))
user2id = {j:i for i, j in id2user.items()}
id2book = dict(enumerate(transaction.책제목.unique()))
book2id = {j:i for i, j in id2book.items()}
id2cat = dict(enumerate(transaction.카테고리.unique()))
cat2id = {j:i for i, j in id2cat.items()}

In [4]:
data = transaction[['회원번호', '책제목']]
data.회원번호 = data.회원번호.map(lambda x: user2id[x])
data.책제목 = data.책제목.map(lambda x: book2id[x])
data = data.drop_duplicates()

C:\Users\KwonYujin\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
data_cat = transaction.groupby('책제목')['카테고리'].unique().map(lambda x: x[0]).reset_index()

In [6]:
data_cat['책제목'] = data_cat.책제목.map(lambda x: book2id[x])
data_cat['카테고리'] = data_cat.카테고리.map(lambda x: cat2id[x])

In [7]:
data_cat = dict(data_cat.values)

In [8]:
data_cat

{49680: 2,
 59354: 6,
 35546: 10,
 34100: 4,
 6822: 13,
 16150: 15,
 60654: 18,
 61015: 9,
 54755: 17,
 53892: 0,
 27023: 20,
 4934: 5,
 59994: 0,
 63200: 20,
 63201: 20,
 63212: 20,
 52054: 20,
 35465: 20,
 54835: 22,
 54103: 13,
 60389: 20,
 60391: 20,
 60390: 20,
 57558: 20,
 3562: 5,
 32690: 7,
 14375: 4,
 15537: 4,
 25169: 5,
 37075: 9,
 3534: 10,
 61284: 10,
 3535: 10,
 56406: 10,
 3532: 10,
 3533: 10,
 3536: 10,
 65351: 4,
 11966: 4,
 32516: 9,
 45660: 4,
 40305: 4,
 47160: 4,
 21828: 4,
 53391: 10,
 37820: 15,
 63205: 15,
 65828: 16,
 48852: 14,
 55799: 14,
 51886: 18,
 21146: 18,
 22401: 4,
 14839: 9,
 63622: 20,
 16894: 15,
 20183: 15,
 20182: 15,
 42549: 20,
 3946: 1,
 20518: 10,
 31033: 23,
 19399: 20,
 25855: 16,
 5329: 14,
 61644: 9,
 1284: 0,
 47962: 0,
 20028: 0,
 971: 0,
 15871: 0,
 26366: 0,
 101: 0,
 50547: 0,
 771: 0,
 65747: 1,
 1489: 18,
 31029: 24,
 33416: 24,
 30192: 24,
 33415: 24,
 54712: 24,
 29343: 24,
 29344: 24,
 61976: 24,
 42949: 24,
 24578: 24,
 45788: 

In [17]:
ids = data.groupby('회원번호').책제목.count().reset_index().query("책제목 > 5").회원번호.unique()
data = data.query('회원번호 in @ids')

In [18]:
data

,회원번호,책제목
11,5,11
12,5,12
13,5,13
14,5,14
15,5,15
...,...,...
251144,26883,875
251146,26883,30273
251147,26883,54760
251148,26883,66283


In [19]:
data.to_csv('main_data.csv', index=False)

In [20]:
data_cat = transaction[['회원번호', '카테고리']]
data_cat.회원번호 = data_cat.회원번호.map(lambda x: user2id[x])
data_cat.카테고리 = data_cat.카테고리.map(lambda x: cat2id[x])
data_cat = data_cat.drop_duplicates()

C:\Users\KwonYujin\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
data_tr = pd.read_csv('main_data.csv')

In [22]:
data_tr.shape

(202667, 2)

In [23]:
train_data = data_tr.copy()

In [24]:
items = data.groupby('회원번호')['책제목'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = train_data.append(pd.DataFrame(test_data, columns=['회원번호', '책제목'])).drop_duplicates(keep=False)

In [25]:
train_data.to_csv('main_data_train.csv', index=False)

In [26]:
item_num = transaction.책제목.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [27]:
res

회원번호
5        [21, 64439, 48473, 49783, 15392, 61837, 56292,...
17       [60, 63176, 42924, 728, 43299, 18026, 7320, 18...
35       [104, 31523, 5021, 63429, 36615, 56869, 65330,...
38       [113, 34865, 51582, 33621, 41161, 40915, 32655...
41       [135, 52309, 39254, 29264, 48731, 26436, 45836...
                               ...                        
26876    [6255, 28862, 41067, 40398, 60074, 17509, 1573...
26877    [2281, 35104, 24603, 19362, 35841, 49687, 2853...
26878    [66279, 35514, 49667, 10232, 53546, 10354, 224...
26880    [66282, 31565, 52450, 46041, 31633, 40526, 301...
26883    [44140, 38879, 21269, 37728, 33599, 48072, 365...
Name: 책제목, Length: 11905, dtype: object

In [28]:
np.save('main_data_test.npy', res.reset_index().values, allow_pickle=True)

In [29]:
test_data_raw = np.load('main_data_test.npy', allow_pickle=True)

In [30]:
test_data = []
for data in test_data_raw:
    u = data[0]
    i = data[1]
    for item in i:
        test_data.append([u, int(item)])

In [34]:
print(data_tr.회원번호.nunique(), data_tr.책제목.nunique())

11905 60037
